# Classification of messages as spam or not spam using Naive Bayes algorithm

**Import Dataset - upload the SMS text file to the content folder on the left panel before running**

In [ ]:
import pandas as pd
import numpy as np

# Import Dataset - upload the SMS text file to the content folder on the left panel before running
df = pd.read_table('SMS.txt', sep='\t', header=None, names=['label', 'sms_message'])
df

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
# map the 'ham' value to 0 and the 'spam' value to 1.
df['label_binary'] = df.label.map({'ham':0,'spam':1})
df.head()

,label,sms_message,label_binary
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [ ]:
# Get stats
df['label'].value_counts()

,count
label,
ham,4825
spam,747


In [ ]:
#  data cleaning
df['sms_message'] = df['sms_message'].str.replace(r'[\W_]+', ' ', regex=True).str.strip() # Removes punctuation and leading/trailing spaces
df['sms_message'] = df['sms_message'].str.lower() ### making all the words lowercase
df.head(10)

,label,sms_message,label_binary
0,ham,go until jurong point crazy available only in ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry in 2 a wkly comp to win fa cup fina...,1
3,ham,u dun say so early hor u c already then say,0
4,ham,nah i don t think he goes to usf he lives arou...,0
5,spam,freemsg hey there darling it s been 3 week s n...,1
6,ham,even my brother is not like to speak with me t...,0
7,ham,as per your request melle melle oru minnaminun...,0
8,spam,winner as a valued network customer you have b...,1
9,spam,had your mobile 11 months or more u r entitled...,1


In [ ]:
# Randomly shuffle the records in the dataset to avoid bias
df = df.sample(frac=1, random_state=1)
df.head(10)

,label,sms_message,label_binary
1078,ham,yep by the pretty sculpture,0
4028,ham,yes princess are you going to make me moan,0
958,ham,welp apparently he retired,0
4642,ham,havent,0
4674,ham,i forgot 2 ask ü all smth there s a card on da...,0
5461,ham,ok i thk i got it then u wan me 2 come now or wat,0
4210,ham,i want kfc its tuesday only buy 2 meals only 2...,0
4216,ham,no dear i was sleeping p,0
1603,ham,ok pa nothing problem,0
1504,ham,ill be there on lt gt ok,0


In [ ]:
# Split into training and test sets
training_test_index = round(len(df) * 0.8)

training = df[:training_test_index].reset_index(drop=True)
test = df[training_test_index:].reset_index(drop=True)

print('-- Training set stats --')
print(training.shape)
print(training['label_binary'].value_counts())
print('-- Test set stats --')
print(test.shape)
print(test['label_binary'].value_counts())

-- Training set stats --
(4458, 3)
label_binary
0    3858
1     600
Name: count, dtype: int64
-- Test set stats --
(1114, 3)
label_binary
0    967
1    147
Name: count, dtype: int64


In [ ]:
### creating vocabulary from training data
training['sms_message'] = training['sms_message'].str.split()
vocabulary = []
for sms in training['sms_message']:
   for word in sms:
      vocabulary.append(word)
vocabulary = list(set(vocabulary))  ### only count the number of unique words
print(len(vocabulary))
vocabulary[0:9]

7780


['153',
 'started',
 'phones',
 'swan',
 'cry',
 'erupt',
 'taught',
 'empty',
 'nigpun']

In [ ]:
word_counts_per_sms = {unique_word: [0] * len(training['sms_message']) for unique_word in vocabulary}

for index, sms in enumerate(training['sms_message']):
   for word in sms:
      word_counts_per_sms[word][index] += 1
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts

,153,started,phones,swan,cry,erupt,taught,empty,nigpun,forget,...,performed,symptoms,78,knackered,kisi,69969,eh74rr,teresa,slots,mad
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4456,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
training_new = pd.concat([training, word_counts], axis=1)
training_new.head()

,label,sms_message,label_binary,153,started,phones,swan,cry,erupt,taught,...,performed,symptoms,78,knackered,kisi,69969,eh74rr,teresa,slots,mad
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Run a baseline model evaluation
# Set all 'predicted to 0 or 1 randomly to get a baseline (coin-flip)
test['predicted'] = np.random.randint(0, 2, size=len(test))
test['predicted'].value_counts()

,count
predicted,
0,572
1,542


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('Accuracy score: {}'.format(accuracy_score(test['label_binary'], test['predicted'])))
print('Precision score: {}'.format(precision_score(test['label_binary'], test['predicted'])))
print('Recall score: {}'.format(recall_score(test['label_binary'], test['predicted'])))
print('F1 score: {}'.format(f1_score(test['label_binary'], test['predicted'])))

Accuracy score: 0.5233393177737882
Precision score: 0.14575645756457564
Recall score: 0.5374149659863946
F1 score: 0.22931785195936139


## ** Custom implementation starts here**
prediction result is saved into the column `test['predicted']` for the evaludation to run automatically.  


In [ ]:
# Laplace smoothing
alpha = 1

p_spam = training['label_binary'].sum() / len(training)
p_ham = 1 - p_spam

print(f'P(Spam) = {p_spam:.4f}')
print(f'P(Ham) = {p_ham:.4f}')

P(Spam) = 0.1346
P(Ham) = 0.8654


In [ ]:
# Step 1: caculate P(Spam) and P(Ham)
# Step 2: count N_Spam, N_Ham
# Step 3: count the number of times the word w occurs in spam/ham message: N_w_spam, N_w_ham
# Step 4: then calculate the prob of occurance of each word:
#         p(w|spam)=(N_w_spam+alpha)/(N_Spam+alpha*N_Vocabulary)
#         p(w|Ham)=(N_w_ham+alpha)/(N_Ham+alpha*N_Vocabulary)
# Step 5: perform the prediction on the test dataset messages using the Naiive Bayes method. Store your prediction results (1=spam or 0=ham ) to test['predicted']
# Step 6: Summarize the results in a confusion matrix and print out the four values of the confusion matrix
#         Verify that printout is consistent with the output from test['label_binary'].value_counts() and test['predicted'].value_counts()

#--------------------------------------------------------------------------------------------------------------------------------------------------------------
# Step 2: Count total words in spam and ham messages
spam_messages = training_new[training_new['label_binary'] == 1]
ham_messages = training_new[training_new['label_binary'] == 0]

# N_Spam: total number of words in all spam messages
N_spam = spam_messages[vocabulary].sum().sum()
# N_Ham: total number of words in all ham messages
N_ham = ham_messages[vocabulary].sum().sum()

print(f'Total words in spam messages: {N_spam}')
print(f'Total words in ham messages: {N_ham}')
print(f'Vocabulary size: {len(vocabulary)}')

# Step 3 & 4: Calculate P(w|spam) and P(w|ham) for each word
p_word_spam = {}
p_word_ham = {}

for word in vocabulary:
    N_w_spam = spam_messages[word].sum()
    N_w_ham = ham_messages[word].sum()

    p_word_spam[word] = (N_w_spam + alpha) / (N_spam + alpha * len(vocabulary))
    p_word_ham[word] = (N_w_ham + alpha) / (N_ham + alpha * len(vocabulary))

print(f'\\nCalculated probabilities for {len(vocabulary)} words')

# Step 5: Predict spam/ham for test messages
def classify_message(message):
    """
    Classify a message as spam (1) or ham (0) using Naive Bayes
    """
    # Start with prior probabilities (in log space to avoid underflow)
    log_p_spam = np.log(p_spam)
    log_p_ham = np.log(p_ham)

    # Split message into words
    words = message.lower().split()

    # Multiply probabilities for each word (add logs)
    for word in words:
        if word in p_word_spam:
            log_p_spam += np.log(p_word_spam[word])
            log_p_ham += np.log(p_word_ham[word])

    # Return 1 for spam, 0 for ham
    return 1 if log_p_spam > log_p_ham else 0

# Apply classification to test set
test['predicted'] = test['sms_message'].apply(classify_message)

print('\\nPrediction complete!')
print(test['predicted'].value_counts())

Total words in spam messages: 15193
Total words in ham messages: 57233
Vocabulary size: 7780
\nCalculated probabilities for 7780 words
\nPrediction complete!
predicted
0    970
1    144
Name: count, dtype: int64


In [ ]:
# Step 6: Create and display confusion matrix
from sklearn.metrics import confusion_matrix

# Calculate confusion matrix
cm = confusion_matrix(test['label_binary'], test['predicted'])

# Extract values
tn, fp, fn, tp = cm.ravel()

print('\\n CONFUSION MATRIX ')
print(f'                Predicted Ham  Predicted Spam')
print(f'Actual Ham      {tn:6d}         {fp:6d}')
print(f'Actual Spam     {fn:6d}         {tp:6d}')
print()
print(f'True Negatives (TN):  {tn}  (Correctly predicted ham)')
print(f'False Positives (FP): {fp}  (Ham predicted as spam)')
print(f'False Negatives (FN): {fn}  (Spam predicted as ham)')
print(f'True Positives (TP):  {tp}  (Correctly predicted spam)')
print()
print(f'Total test messages: {tn + fp + fn + tp}')
print(f'Actual ham messages: {tn + fp}')
print(f'Actual spam messages: {fn + tp}')

\n CONFUSION MATRIX 
                Predicted Ham  Predicted Spam
Actual Ham         962              5
Actual Spam          8            139

True Negatives (TN):  962  (Correctly predicted ham)
False Positives (FP): 5  (Ham predicted as spam)
False Negatives (FN): 8  (Spam predicted as ham)
True Positives (TP):  139  (Correctly predicted spam)

Total test messages: 1114
Actual ham messages: 967
Actual spam messages: 147


**Evaluate your implementation** for accuracy, precision, recall and F1_score.  The performance points of your implementation will be calculated automatically.  However, it is only awarded if the predictions are made by a Naive Bayes implementation.

**30 points** for how well your implementation predicts spam.  A correct implementation should achieve an F1 score above 0.90.  
## **DO NOT modify this cell below.**

In [ ]:
# Model Evaluation
print('Accuracy score: {}'.format(accuracy_score(test['label_binary'], test['predicted'])))
print('Precision score: {}'.format(precision_score(test['label_binary'], test['predicted'])))
print('Recall score: {}'.format(recall_score(test['label_binary'], test['predicted'])))
my_f1_score = f1_score(test['label_binary'], test['predicted'])
print('F1 score: {}'.format(my_f1_score))
performance_point = round(np.clip((my_f1_score - 0.20) / (0.9-0.20) * 30, 0, 30))
print('Your perforamnce point: {}'.format(performance_point))

Accuracy score: 0.9883303411131059
Precision score: 0.9652777777777778
Recall score: 0.9455782312925171
F1 score: 0.9553264604810997
Your perforamnce point: 30


**Analysis of implementation of the Naive Bayes algorithm:**

In [ ]:

# True Negative (correctly predicted ham)
tn_example = test[(test['label_binary'] == 0) & (test['predicted'] == 0)].iloc[0]
# False Positive (ham predicted as spam)
fp_example = test[(test['label_binary'] == 0) & (test['predicted'] == 1)].iloc[0]
# False Negative (spam predicted as ham)
fn_example = test[(test['label_binary'] == 1) & (test['predicted'] == 0)].iloc[0]
# True Positive (correctly predicted spam)
tp_example = test[(test['label_binary'] == 1) & (test['predicted'] == 1)].iloc[0]

def analyze_prediction(message, actual_label):
    """Show detailed probability calculations for a message"""
    words = message.lower().split()

    log_p_spam = np.log(p_spam)
    log_p_ham = np.log(p_ham)

    print(f'Message: "{message[:80]}..."')
    print(f'Actual label: {"SPAM" if actual_label == 1 else "HAM"}')
    print(f'\\nInitial probabilities:')
    print(f'  log P(Spam) = {log_p_spam:.4f}')
    print(f'  log P(Ham) = {log_p_ham:.4f}')
    print(f'\\nTop 5 most influential words:')

    word_contributions = []
    for word in words:
        if word in p_word_spam:
            spam_contrib = np.log(p_word_spam[word])
            ham_contrib = np.log(p_word_ham[word])
            diff = abs(spam_contrib - ham_contrib)
            word_contributions.append((word, spam_contrib, ham_contrib, diff))
            log_p_spam += spam_contrib
            log_p_ham += ham_contrib

    # Sort by contribution difference
    word_contributions.sort(key=lambda x: x[3], reverse=True)
    for word, spam_c, ham_c, _ in word_contributions[:5]:
        print(f'  "{word}": log P(w|spam)={spam_c:.4f}, log P(w|ham)={ham_c:.4f}')

    print(f'\\nFinal probabilities:')
    print(f'  log P(Spam|message) = {log_p_spam:.4f}')
    print(f'  log P(Ham|message) = {log_p_ham:.4f}')
    print(f'  Prediction: {"SPAM" if log_p_spam > log_p_ham else "HAM"}')
    print('='*80)

print('\\n TRUE NEGATIVE (Correctly classified as HAM)')
analyze_prediction(tn_example['sms_message'], tn_example['label_binary'])

print('\\n FALSE POSITIVE (HAM misclassified as SPAM) ')
analyze_prediction(fp_example['sms_message'], fp_example['label_binary'])

print('\\n FALSE NEGATIVE (SPAM misclassified as HAM) ')
analyze_prediction(fn_example['sms_message'], fn_example['label_binary'])

print('\\n TRUE POSITIVE (Correctly classified as SPAM) ')
analyze_prediction(tp_example['sms_message'], tp_example['label_binary'])

\n### TRUE NEGATIVE (Correctly classified as HAM) ###
Message: "later i guess i needa do mcat study too..."
Actual label: HAM
\nInitial probabilities:
  log P(Spam) = -2.0055
  log P(Ham) = -0.1446
\nTop 5 most influential words:
  "later": log P(w|spam)=-10.0421, log P(w|ham)=-6.4872
  "i": log P(w|spam)=-6.1102, log P(w|ham)=-3.2983
  "i": log P(w|spam)=-6.1102, log P(w|ham)=-3.2983
  "too": log P(w|spam)=-9.3489, log P(w|ham)=-6.5715
  "do": log P(w|spam)=-6.8640, log P(w|ham)=-5.3266
\nFinal probabilities:
  log P(Spam|message) = -58.6193
  log P(Ham|message) = -42.3007
  Prediction: HAM
\n### FALSE POSITIVE (HAM misclassified as SPAM) ###
Message: "unlimited texts limited minutes..."
Actual label: HAM
\nInitial probabilities:
  log P(Spam) = -2.0055
  log P(Ham) = -0.1446
\nTop 5 most influential words:
  "texts": log P(w|spam)=-7.0976, log P(w|ham)=-9.4729
  "unlimited": log P(w|spam)=-7.7395, log P(w|ham)=-9.6960
  "minutes": log P(w|spam)=-8.4326, log P(w|ham)=-7.9913
\nFinal p


**False Positive (Ham predicted as Spam):**
This misclassification occurs when a legitimate message contains words that are strongly associated with spam in the training data. For example, words like "free", "win", "call", or "text" might appear in both spam and legitimate messages, but have higher probability in spam messages. The Naive Bayes algorithm multiplies these probabilities together, and if enough spam-like words appear, the overall spam probability exceeds the ham probability, even though the message is legitimate.

**False Negative (Spam predicted as Ham):**
This occurs when spam messages are written in a way that mimics normal conversation or uses words not commonly found in the training spam data. Some spam messages intentionally avoid typical spam keywords to evade detection. Additionally, if the spam message is very short or uses unconventional spelling/slang that wasn't present in the training vocabulary, the algorithm may not have strong evidence to classify it as spam, leading to misclassification as ham.

The assumption that all words are independent can also contribute to both types of errors, as context and word combinations matter in natural language but are not captured by this model.

**Bonus:**  Use function MultinomialNB (from sklearn.naive_bayes import MultinomialNB) to perform the same classification and evaludate its results.

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

X_train = training_new[vocabulary]
y_train = training_new['label_binary']

test['sms_message'] = test['sms_message'].str.split()
test_word_counts = {word: [0] * len(test) for word in vocabulary}

for index, message in enumerate(test['sms_message']):
    for word in message:
        if word in test_word_counts:
            test_word_counts[word][index] += 1

X_test = pd.DataFrame(test_word_counts)
y_test = test['label_binary']

# Training MultinomialNB classifier
nb_classifier = MultinomialNB(alpha=1.0)
nb_classifier.fit(X_train, y_train)

# Make predictions
sklearn_predictions = nb_classifier.predict(X_test)

# Evaluate
print(' sklearn MultinomialNB Results ')
print(f'Accuracy: {accuracy_score(y_test, sklearn_predictions):.4f}')
print(f'Precision: {precision_score(y_test, sklearn_predictions):.4f}')
print(f'Recall: {recall_score(y_test, sklearn_predictions):.4f}')
print(f'F1 Score: {f1_score(y_test, sklearn_predictions):.4f}')

# Confusion matrix
cm_sklearn = confusion_matrix(y_test, sklearn_predictions)
tn, fp, fn, tp = cm_sklearn.ravel()
print(f'\\nConfusion Matrix:')
print(f'TN: {tn}, FP: {fp}, FN: {fn}, TP: {tp}')

 sklearn MultinomialNB Results 
Accuracy: 0.9883
Precision: 0.9653
Recall: 0.9456
F1 Score: 0.9553
\nConfusion Matrix:
TN: 962, FP: 5, FN: 8, TP: 139
